In [53]:
import torch
from torch import nn
from torchtext.datasets import IMDB
from torchtext.vocab import GloVe
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data import BucketIterator



ImportError: cannot import name 'BucketIterator' from 'torchtext.data' (/usr/local/lib/python3.10/dist-packages/torchtext/data/__init__.py)

In [2]:
train_iter = IMDB(split='train')
test_iter = IMDB(split='test')

In [3]:
# def tokenize(line):
#     return line.split()

def process_text_iter(text_iter):
    texts, labels = [], []
    for label, line in text_iter:
        texts.append(line)
        labels.append(label)
    return texts, labels
    #     yield {'text': tokenize(line), 'label': label}

train_text, train_label = process_text_iter(train_iter)
test_text, test_label = process_text_iter(test_iter)


# train = []
# for label, line in train_iter:
#     train.append(
#         {'text': tokenize(line), 'label': label}
#     )
# train[0]


In [8]:
tokenizer = get_tokenizer('basic_english')

train_tokenized_texts = [tokenizer(text) for text in train_text]
test_tokenized_texts = [tokenizer(text) for text in test_text]

In [32]:
def yield_tokens(data_iter):
    for tokens in data_iter:
        yield tokens
    
vocab = build_vocab_from_iterator(yield_tokens(train_tokenized_texts), max_tokens=25000, specials=["<unk>", '<pad>'])
vocab.set_default_index(vocab["<unk>"])

In [33]:
glove = GloVe(name='6B', dim=100)

In [43]:
embedding_vectors = torch.zeros(len(vocab), glove.dim)
for i, word in enumerate(vocab.get_itos()):  # get_itos() returns the list of words in vocab
    if word in glove.stoi:  # stoi -> String-to-Index
        embedding_vectors[i] = glove[word]

In [84]:
train_indexed_sequences = [[vocab[token] for token in tokens] for tokens in train_tokenized_texts]
test_indexed_sequences = [[vocab[token] for token in tokens] for tokens in test_tokenized_texts]

In [93]:
import torch.nn.utils.rnn as rnn_utils

max_length = max(len(seq) for seq in train_indexed_sequences)
padded_sequences = rnn_utils.pad_sequence(
    [torch.tensor(seq) for seq in train_indexed_sequences],
    batch_first=True,
    padding_value=vocab["<pad>"]
)
padded_sequences

sequence_lengths = torch.tensor([len(seq) for seq in train_indexed_sequences])
sequence_lengths

tensor([317, 254, 101,  ..., 654, 159, 494])

In [148]:
class RNN(nn.Module):

    def __init__(
        self, 
        # vocab_size, 
        # embedding_dim, 
        embedding_vectors,
        hidden_size,
        pad_idx
    ):
        
        super(RNN, self).__init__()

        self.embedding = nn.Embedding.from_pretrained(
            embedding_vectors, 
            padding_idx=pad_idx
        )
        self.lstm = nn.LSTM(
            self.embedding.weight.shape[1], 
            hidden_size, 
            batch_first=True
        )
        self.fc = nn.Linear(
            hidden_size, 
            2
        )

    def forward(self, text, text_lengths):
        print(text.shape)
        embedded = self.embedding(text)
        print(embedded.shape)

        packed_input = rnn_utils.pack_padded_sequence(
            embedded, text_lengths, batch_first=True, enforce_sorted=False
        )
        print(packed_input.batch_sizes)

        packed_output, (hidden, cell) = self.lstm(packed_input)

        print(packed_output.batch_sizes)

        print(hidden.shape)
        output, _ = rnn_utils.pad_packed_sequence(packed_output, batch_first=True)
        print(output.shape)

        return output
pad_idx = '<pad>'
model = RNN(
    embedding_vectors=embedding_vectors,
    hidden_size=256,
    pad_idx=vocab[pad_idx]
)

model    
data = model(padded_sequences[:1000], sequence_lengths[:1000])


torch.Size([1000, 1722])
torch.Size([1000, 1722, 100])
tensor([1000, 1000, 1000,  ...,    1,    1,    1])
tensor([1000, 1000, 1000,  ...,    1,    1,    1])
torch.Size([1, 1000, 256])
torch.Size([1000, 1212, 256])


In [146]:
data[0].shape

torch.Size([1212, 256])

In [128]:
len(train_tokenized_texts[2])

101